In [2]:
#Python File for ETL Project

In [3]:
import pandas as pd
from sqlalchemy import create_engine

In [4]:
#Extract CSV's into DataFrames

In [5]:
# File Legend:

# CSV_1: Netflix Titles
# CSV_2: Metascore Data 


In [24]:
# Extract Step
# Netflix Titles

CSV_1 = "Resources/netflix_titles.csv"
netflix_df = pd.read_csv(CSV_1)
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [7]:
# List total columns & Identify columns for revised dataframe

for col in netflix_df.columns: 
    print(col) 
    

show_id
type
title
director
cast
country
date_added
release_year
rating
duration
listed_in
description


In [20]:
# Transform Step
# Revise Dataset with select columns

revised_netflix_df = netflix_df[["type","title","director","country","release_year","listed_in","description"]]
revised_netflix_df.head()


,type,title,director,country,release_year,listed_in,description
0,TV Show,3%,NaN,Brazil,2020,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,Movie,7:19,Jorge Michel Grau,Mexico,2016,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,Movie,23:59,Gilbert Chan,Singapore,2011,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,Movie,9,Shane Acker,United States,2009,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,Movie,21,Robert Luketic,United States,2008,Dramas,A brilliant group of students become card-coun...


In [21]:
# Transform Step
# Drop TV Type 
drop_netflix_df = revised_netflix_df[revised_netflix_df.type != 'TV Show']
drop_netflix_df.head()

,type,title,director,country,release_year,listed_in,description
1,Movie,7:19,Jorge Michel Grau,Mexico,2016,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,Movie,23:59,Gilbert Chan,Singapore,2011,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,Movie,9,Shane Acker,United States,2009,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,Movie,21,Robert Luketic,United States,2008,Dramas,A brilliant group of students become card-coun...
6,Movie,122,Yasir Al Yasiri,Egypt,2019,"Horror Movies, International Movies","After an awful accident, a couple admitted to ..."


In [22]:
# Transform Step
# Rename columns
renamed_netflix_df = drop_netflix_df.rename(columns={"title":"movie_title","country":"produced_in","listed_in": "netflix_genre"})
renamed_netflix_df.head()

,type,movie_title,director,produced_in,release_year,netflix_genre,description
1,Movie,7:19,Jorge Michel Grau,Mexico,2016,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,Movie,23:59,Gilbert Chan,Singapore,2011,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,Movie,9,Shane Acker,United States,2009,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,Movie,21,Robert Luketic,United States,2008,Dramas,A brilliant group of students become card-coun...
6,Movie,122,Yasir Al Yasiri,Egypt,2019,"Horror Movies, International Movies","After an awful accident, a couple admitted to ..."


In [25]:
# Transform Step
# Remove Type Column
movie_netflix_df = renamed_netflix_df.drop('type', axis=1)
movie_netflix_df.head()

,movie_title,director,produced_in,release_year,netflix_genre,description
1,7:19,Jorge Michel Grau,Mexico,2016,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,23:59,Gilbert Chan,Singapore,2011,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,9,Shane Acker,United States,2009,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,21,Robert Luketic,United States,2008,Dramas,A brilliant group of students become card-coun...
6,122,Yasir Al Yasiri,Egypt,2019,"Horror Movies, International Movies","After an awful accident, a couple admitted to ..."


In [26]:
# Transform Step 
# split produced_in
# Define 1st value in listed_in column as main country the movie was produced in
produced_df = movie_netflix_df

# split genre column
split_produced_df = produced_df['produced_in'].str.split(',', 1, expand=True)

# Rename columns
renamed_produced_df = split_produced_df.rename(columns={0:"primary_country",1:"other_countries"})
renamed_produced_df.head()

# drop column
country_df = renamed_produced_df.drop('other_countries', axis=1)
country_df.head()


,primary_country
1,Mexico
2,Singapore
3,United States
4,United States
6,Egypt


In [27]:
# Get Unique Values in 
country_list = country_df['primary_country'].unique()
country_list

array(['Mexico', 'Singapore', 'United States', 'Egypt', 'India',
       'Thailand', 'Nigeria', 'Norway', 'United Kingdom', 'South Korea',
       'Italy', 'Canada', 'Indonesia', 'Romania', 'Spain', 'Turkey',
       'Iceland', 'South Africa', 'France', 'Portugal', nan, 'Hong Kong',
       'Argentina', 'Germany', 'Denmark', 'Poland', 'Japan', 'Kenya',
       'New Zealand', 'Pakistan', 'Australia', 'China', 'Taiwan',
       'Netherlands', 'Philippines', 'United Arab Emirates', 'Brazil',
       'Iran', 'Israel', 'Uruguay', 'Bulgaria', 'Colombia',
       'Soviet Union', 'Sweden', 'Malaysia', 'Ireland', 'Serbia', 'Peru',
       'Chile', 'Ghana', 'Saudi Arabia', 'Namibia', 'Lebanon', 'Belgium',
       'Vietnam', 'Russia', 'Kuwait', 'Czech Republic', 'Zimbabwe',
       'Hungary', 'Finland', 'Venezuela', 'Cambodia', 'West Germany',
       'Slovenia', 'Switzerland', 'Austria', 'Bangladesh', 'Georgia',
       'Guatemala', 'Jamaica', 'Greece', 'Paraguay', 'Somalia', 'Croatia'],
      dtype=object)

In [28]:
# Classify International Movies 
# might need to be done by merging country_df with movie_netflix_df and then grouping US films... 


In [29]:
# next steps: - JP Can do 
# filter type to just movies 
# remove description column
# rename listed_in to 'Netflix Genre': extract first listed_in value as Netflix Genre
# could filter out international movies, country to United States 

In [30]:
# split neftlix_genre
# Define 1st value in listed_in column as main country the movie was produced in
genre_df = movie_netflix_df

# split genre column
split_genre_df = genre_df['netflix_genre'].str.split(',', 1, expand=True)

# Rename columns
renamed_genre_df = split_genre_df.rename(columns={0:"primary_genre",1:"other_genres"})

# drop column
drop_netflix_genre_df = renamed_genre_df.drop('other_genres', axis=1)

# Rename columns
netflix_genre_df = drop_netflix_genre_df.rename(columns={"primary_genre":"listed_in_netflix"})
netflix_genre_df.head()

,listed_in_netflix
1,Dramas
2,Horror Movies
3,Action & Adventure
4,Dramas
6,Horror Movies


In [31]:
# Extract Step
# Metascore Data

CSV_2 = "Resources/metacritic_movies.csv"
metascore_df = pd.read_csv(CSV_2)
metascore_df.head()

,movie_title,release_date,genre,meta_mixed,meta_negative,meta_positive,metascore,user_mixed,user_negative,user_positive,userscore
0,Anatomy of a Murder,1-Jul-59,"Drama,Mystery,Thriller,Crime",0,0,15,95,0,0,3,tbd
1,Bringing Up Baby,18-Feb-38,"Comedy,Romance,Family",0,1,16,91,1,0,2,tbd
2,After Life,12-May-99,"Drama,Fantasy",0,0,19,91,0,2,1,tbd
3,Gavagai,3-Aug-18,Drama,1,0,6,91,0,1,2,tbd
4,The Hustler,25-Sep-61,"Drama,Sport",1,0,17,90,0,0,3,tbd


In [32]:
#List total columns & identify columns for revised dataframe
for col in metascore_df.columns: 
    print(col) 

movie_title
release_date
genre
meta_mixed
meta_negative
meta_positive
metascore
user_mixed
user_negative
user_positive
userscore


In [33]:
# Transform Step
# Revise Dataset with select columns

revised_metascore_df = metascore_df[["movie_title","genre","metascore"]]
revised_metascore_df.head()


,movie_title,genre,metascore
0,Anatomy of a Murder,"Drama,Mystery,Thriller,Crime",95
1,Bringing Up Baby,"Comedy,Romance,Family",91
2,After Life,"Drama,Fantasy",91
3,Gavagai,Drama,91
4,The Hustler,"Drama,Sport",90


In [34]:
# Transform Step
# Rename columns
renamed_metascore_df = revised_metascore_df.rename(columns={"genre": "metacritic_genre"})
renamed_metascore_df.head()


,movie_title,metacritic_genre,metascore
0,Anatomy of a Murder,"Drama,Mystery,Thriller,Crime",95
1,Bringing Up Baby,"Comedy,Romance,Family",91
2,After Life,"Drama,Fantasy",91
3,Gavagai,Drama,91
4,The Hustler,"Drama,Sport",90


In [35]:
# split metacritic_genre
# Define 1st value in listed_in column as main country the movie was produced in
metagenre_df = renamed_metascore_df[['movie_title','metacritic_genre']]
metagenre_df.head()

,movie_title,metacritic_genre
0,Anatomy of a Murder,"Drama,Mystery,Thriller,Crime"
1,Bringing Up Baby,"Comedy,Romance,Family"
2,After Life,"Drama,Fantasy"
3,Gavagai,Drama
4,The Hustler,"Drama,Sport"


In [36]:
# split genre column
split_metagenre_df = renamed_metascore_df['metacritic_genre'].str.split(',', 1, expand=True)


In [37]:

# Rename columns
renamed_metagenre_df = split_metagenre_df.rename(columns={0:"primary_genre",1:"other_genres"})

# drop column
drop_genre_df = renamed_metagenre_df.drop('other_genres', axis=1)

# Rename column
metacritic_genre_df = drop_genre_df.rename(columns={"primary_genre":"metacritic_genre"})
metacritic_genre_df.head()

,metacritic_genre
0,Drama
1,Comedy
2,Drama
3,Drama
4,Drama


In [38]:
new_metacritic_df = pd.merge(renamed_metascore_df, metacritic_genre_df, on="metacritic_genre")
new_metacritic_df.head()

,movie_title,metacritic_genre,metascore
0,Gavagai,Drama,91
1,Gavagai,Drama,91
2,Gavagai,Drama,91
3,Gavagai,Drama,91
4,Gavagai,Drama,91


In [39]:
# Binning by score range 
#score range legend (4 star range)

# 4-Star Scale
# Their Grade	Converts to
# 4- 100
# 3 -75
# 2 - 50
# 1 - 25
# 0 - 0

 # Create bins in which to place values based upon TED Talk views
bins = [0, 25, 50, 75, 100]

# Create labels for these bins
group_labels = ["1 star", "2 stars", "3 stars", "4 stars"]

# Place the data series into a new column inside of the DataFrame
renamed_metascore_df["Rating"] = pd.cut(renamed_metascore_df["metascore"], bins, labels=group_labels)
bin_df = renamed_metascore_df
bin_df.head()

,movie_title,metacritic_genre,metascore,Rating
0,Anatomy of a Murder,"Drama,Mystery,Thriller,Crime",95,4 stars
1,Bringing Up Baby,"Comedy,Romance,Family",91,4 stars
2,After Life,"Drama,Fantasy",91,4 stars
3,Gavagai,Drama,91,4 stars
4,The Hustler,"Drama,Sport",90,4 stars


In [40]:
# Groupby
# Create a GroupBy object based upon "View Group"
new_bin_df = bin_df.groupby("Rating")
new_bin_df.head(2)

,movie_title,metacritic_genre,metascore,Rating
0,Anatomy of a Murder,"Drama,Mystery,Thriller,Crime",95,4 stars
1,Bringing Up Baby,"Comedy,Romance,Family",91,4 stars
31,Heavy,"Drama,Romance",75,3 stars
32,Charlie's Country,Drama,75,3 stars
155,Burn Your Maps,Adventure,50,2 stars
156,Six Dance Lessons in Six Weeks,"Drama,Comedy",50,2 stars
212,Chloe and Theo,"Drama,Comedy",24,1 star
213,30 Beats,"Comedy,Romance",20,1 star


In [41]:
# Sort Descending metascore
bin_df.sort_values(by='metascore', ascending=False)
bin_df.head()

,movie_title,metacritic_genre,metascore,Rating
0,Anatomy of a Murder,"Drama,Mystery,Thriller,Crime",95,4 stars
1,Bringing Up Baby,"Comedy,Romance,Family",91,4 stars
2,After Life,"Drama,Fantasy",91,4 stars
3,Gavagai,Drama,91,4 stars
4,The Hustler,"Drama,Sport",90,4 stars


In [43]:
# attempt to merge netflix & metascore dfs

merge_movie_df = pd.merge(movie_netflix_df,bin_df, on='movie_title')
merge_movie_df.head()

,movie_title,director,produced_in,release_year,netflix_genre,description,metacritic_genre,metascore,Rating
0,9,Shane Acker,United States,2009,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi...","Adventure,Sci-Fi,Drama,Fantasy,Animation",60,3 stars
1,21,Robert Luketic,United States,2008,Dramas,A brilliant group of students become card-coun...,Drama,48,2 stars
2,1922,Zak Hilditch,United States,2017,"Dramas, Thrillers",A farmer pens a confession admitting to his wi...,"Drama,Mystery,Thriller,Horror,Crime",70,3 stars
3,22-Jul,Paul Greengrass,"Norway, Iceland, United States",2018,"Dramas, Thrillers","After devastating terror attacks in Norway, a ...","Biography,Drama,History,Thriller,Crime",69,3 stars
4,Æon Flux,Karyn Kusama,United States,2005,"Action & Adventure, Sci-Fi & Fantasy","Aiming to hasten an uprising, the leader of an...","Action,Adventure,Sci-Fi,Thriller",36,2 stars


In [44]:
# Sort Descending metascore
sort_merge_df = merge_movie_df.sort_values(by='metascore', ascending=False)
use_this_df = sort_merge_df
use_this_df.head()

,movie_title,director,produced_in,release_year,netflix_genre,description,metacritic_genre,metascore,Rating
578,Moonlight,Barry Jenkins,United States,2016,"Dramas, Independent Movies, LGBTQ Movies","In a crime-infested Miami neighborhood, a gay ...",Drama,99,4 stars
652,Pan's Labyrinth,Guillermo del Toro,"Mexico, Spain",2006,"Dramas, International Movies, Sci-Fi & Fantasy",Young Ofelia meets a mythical faun who claims ...,"Drama,Mystery,Thriller,Fantasy,War",98,4 stars
687,Psycho,Mysskin,India,2020,"Horror Movies, International Movies, Thrillers",As a visually impaired man attempts to rescue ...,"Mystery,Thriller,Horror",97,4 stars
724,Rosemary's Baby,Roman Polanski,United States,1968,"Classic Movies, Horror Movies, Thrillers",A woman is thrilled to find out she's pregnant...,"Drama,Horror",96,4 stars
545,Mean Streets,Martin Scorsese,United States,1973,"Classic Movies, Dramas, Independent Movies","In New York's Little Italy, a low-level hoodlu...","Drama,Thriller,Crime",96,4 stars


In [ ]:
#  Create database connection (need to update.. this is from class example)
# # connection_string = "postgres:postgres@localhost:5432/customer_db"
# engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
#  # Confirm tables
# engine.table_names()

In [ ]:
#  Load DataFrames into database (need to update.. this is from class example)

In [ ]:
#  premise_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True)

In [ ]:
# county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)